In [4]:
%matplotlib inline
import matplotlib
import numpy as np
import omni_decode as omni

sample_rate = 2048000      # sample rate of sdr
samples_per_bit = 50.4185  # This was computed based on previously seen data.

samples = np.fromfile('fsksignal.dat',dtype=np.float32)
packets_offsets = omni.find_offsets(samples,samples_per_bit)

for po in packets_offsets:
    packet_samples = samples[slice(*po)]
    phase = omni.get_phase(packet_samples, samples_per_bit)
    raw_bits = omni.sample_bits(packet_samples, samples_per_bit, phase)
    m_bits = omni.manchester_decode(raw_bits, 'g_e_thomas')
    bits = np.trim_zeros(m_bits + 1) - 1 # Trim leading and trailing errors
    byte_start = omni.find_end_of_preamble(bits)
    bytes = np.packbits(bits[byte_start:])
    print "".join([format(n, '02x') for n in bytes])


5756783e0290577e3e02905660061c020005de08
5756783e029057c03e02905668143a30004c8000000157ff06b76a
5756783e029056823e02905792


In [2]:
preamble = np.unpackbits(np.array([0x57], dtype=np.uint8))
for i in range(0, bits.size):
    if (bits[i:i+8] == preamble).all() and not (bits[i+8:i+16] == preamble).all():
        print "i = %s" % i
        break

bytes = np.packbits(bits[2576:])
print "".join([format(n, '02x') for n in bytes])

i = 2576
5756783e0290577e3e02905660061c020005de08
